# 트위터 감정 분석

이번 실습에서는 트위터 메시지(트윗)의 감정을 파악하는 BERT Classifier를 만들 예정입니다.

감정은 긍정, 부정, 중립의 세 가지 유형으로 구분됩니다.

이를 기준으로 트윗에 담긴 감정을 표시하고, 이 데이터를 사용하여 주어진 트윗의 감정을 분석하는 classifier를 만들겠습니다.

이전 실습에서는 같은 데이터를 사용하여 RNN을 이용하여 classifier를 만들었지만, 이번 실습에서는 BERT를 활용하여 만들 예정입니다.

## 데이터 읽기 

데이터 출처: https://www.kaggle.com/vivekrathi055/sentiment-analysis-on-financial-tweets

`train.csv` 파일을 열어보면 한 라인에 두 개의 열이 있습니다. 
첫 번째 열에는 트위터 메시지인 트윗이 있고 오른쪽에는 태깅된 감정이 있습니다.

- 0: 부정
- 1: 중립
- 2: 긍정

이렇게 세 가지의 감정이 태깅된 것을 알 수 있습니다.

In [ ]:
with open("train.csv") as csv_f:
    head = "\n".join([next(csv_f) for x in range(5)])
print(head)

## 라이브러리 로드

코드 실행에 필요한 라이브러리를 설치하고 로드합니다.

이번 실습에서 사용한 라이브러리는 [huggingface](https://huggingface.co)의 [transformers](https://github.com/huggingface/transformers) 입니다.

In [ ]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim
from sklearn.metrics import classification_report
from transformers import BertModel, BertTokenizer, get_linear_schedule_with_warmup, AdamW
from torch.utils.data import Dataset, DataLoader
import codecs
import csv
import os
from IPython.display import Image

## 모델 클래스 정의

파이토치(PyTorch)를 사용하여 딥러닝 모델의 forward path(순전파)를 정의할 때, 일반적으로 새로운 클래스를 만들기 위해 `nn.Module` 클래스를 상속받아야 합니다.

또한, forward path를 정의하기 위해서는 `forward` 함수를 반드시 정의해야 합니다. `forward` 함수는 모델이 데이터를 입력받아 순전파를 수행하는 메서드입니다.

이번 실습에서는 BERT를 이용한 classifier를 구성하는 것이므로, 적절한 클래스를 작성하였습니다. 이 클래스는 BERT를 활용하여 주어진 트윗의 감정을 분석하는데 적합합니다.

In [ ]:
class BERTClassifier(nn.Module):
    # BERT Classifier 클래스를 정의합니다. Pytorch는 모델을 구성할 때 반드시 nn.Module 클래스를 상속받은 후 이를 토대로 만듭니다.
    def __init__(self, bert_model='bert-base-uncased'):
        # 클래스의 첫 시작인 함수입니다. 여기서 모델에 필요한 여러 변수들을 정의합니다.
        super(BERTClassifier, self).__init__()
        # bert 모델을 불러옵니다. bert 모델은 기학습이 된 것이기에 불러오는 것입니다.
        self.bert = BertModel.from_pretrained(bert_model, return_dict=False)
        self.drop = nn.Dropout(p=0.5)
        hidden_size = self.bert.config.hidden_size
        self.fc = nn.Linear(self.bert.config.hidden_size, 3)

    def forward(self, input_ids, attention_mask):
        # 모델의 forward feed를 수행하는 함수입니다.
        # input_ids와 attention_mask 변수를 입력으로 받아 신경망 모델을 forward 방향으로 탈 때 그 출력을 반환합니다.
        # 단어 id => bert => Dense의 구조입니다.
        _, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)

        text_fea = self.drop(pooled_output)

        # model의 마지막에 classification을 위해 dense layer를 추가합니다.
        text_out = self.fc(text_fea) # Problem 1

        return text_out

## 데이터 클래스 정의

이번 실습에서는 데이터를 불러올 때 BPE로 토큰을 만들 수 있도록 데이터 클래스와 이를 만드는 함수 `make_data_loader`를 정의하겠습니다.

In [ ]:
class BERTDataset(Dataset):
    # pytorch로 데이터를 불러오기 위해서 Dataset 클래스를 상속받아 새로운 클래스를 만듭니다.
    def __init__(self, tweets, labels, tokenizer, max_len):
        # 데이터를 파일로부터 읽어 이를 전달 받습니다.
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.tweets_encodings = list()

        # 하나의 트윗을 BPE tokenizer를 이용하여 토큰을 만듭니다.
        for one_tweet in tweets:
            encoding = self.tokenizer.encode_plus(one_tweet, add_special_tokens=True, max_length=self.max_len,
                                              return_token_type_ids=False, padding='max_length',
                                              return_attention_mask=True, return_tensors='pt', truncation=True)
            input_ids = encoding['input_ids'].flatten()
            attention_mask = encoding['attention_mask'].flatten()
            self.tweets_encodings.append((input_ids, attention_mask))

    def __getitem__(self, idx):
        # 이 클래스에서 하나의 데이터를 뽑을 때 반환하는 값들을 정의합니다.
        encoding = self.tweets_encodings[idx]
        label = self.labels[idx]
        input_ids = encoding[0]
        attention_mask = encoding[1]
        out_label = torch.tensor(int(label), dtype=torch.long)

        # idx번째 데이터에 적합한 input_ids, attention_mask, out_label을 반환합니다.
        # 이를 통해 loader 함수에서 하나씩 데이터를 내놓고 batch 크기만큼 모을 수 있습니다.
        return input_ids, attention_mask, out_label

    def __len__(self):
        return len(self.labels)

In [ ]:
def make_data_loader(dataset_path, tokenizer, max_len, batch_size):
    # DataLoader를 만들어서 데이터를 불러오도록 합니다.
    tweets = list()
    labels = list()
    # 데이터 파일의 내용을 불러와 tweets와 labels 리스트에 넣습니다.
    with codecs.open(dataset_path, "r", "utf-8") as csv_f:
        csv_reader = csv.reader(csv_f)
        for one_row in csv_reader:
            tweets.append(one_row[0])
            labels.append(one_row[-1])
    
    # 앞서 정의한 BERTDataset 클래스에 해당 데이터를 넣습니다.
    ds = BERTDataset(tweets, labels, tokenizer, max_len)

    # 만들어진 BERTDataset 클래스를 DataLoader에 넣고 batch 크기를 전달해줍니다.
    return DataLoader(ds, batch_size=batch_size, num_workers=4)

## train 함수

해당 함수에서는 정의된 `model` 클래스의 인스턴스를 가져와서 이를 train data로 학습시킵니다. 그리고 validation data로 학습 중간에 성능을 평가합니다.

In [ ]:
def train(model, device, optimizer, train_loader, valid_loader, output_file_path, num_epochs):
    # 학습에 필요한 변수들을 기본적으로 정의합니다.
    running_loss = 0.0
    global_step = 0
    train_loss_list = list()
    valid_loss_list = list()
    global_steps_list = list()
    loss_fn = nn.CrossEntropyLoss().to(device)
    best_valid_loss = float("Inf")
    eval_every = 50
    total_steps = len(train_loader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0,
                                                num_training_steps=total_steps)
    
    # model에게 학습이 진행됨을 알려줍니다.
    model.train()
    # num_epochs만큼 epoch을 반복합니다.
    for epoch in range(num_epochs):
        # train_loader를 읽으면 정해진 데이터를 읽어옵니다.
        for input_ids, attention_mask, labels in train_loader:
            # 데이터를 GPU로 옮깁니다.
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            # model을 함수처럼 호출하면 model에서 정의한 forward 함수가 실행됩니다.
            # 즉, 데이터를 모델에 집어넣어 forward방향으로 흐른 후 그 결과를 받습니다.
            output = model(input_ids, attention_mask)

            # forward 결과와 실제 데이터 결과의 차이를 정의한 loss 함수로 구합니다.
            loss = loss_fn(output, labels)

            # 최적화 수행
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

            running_loss += loss.item()
            global_step += 1

            # 50번에 한 번으로 validation 데이터를 이용하여 성능을 검증합니다.
            if global_step % eval_every == 0:
                average_train_loss, average_valid_loss = evaluate(model, device, valid_loader, loss_fn,
                                                                  running_loss, eval_every)
                
                # 검증이 끝난 후 다시 모델에게 학습을 준비시킵니다.
                running_loss = 0.0
                model.train()

                # 결과 출력
                print('Epoch {}, Step {}, Train Loss: {:.4f}, Valid Loss: {:.4f}'
                      .format(epoch + 1, global_step, average_train_loss, average_valid_loss))

                # 결과 저장
                train_loss_list.append(average_train_loss)
                valid_loss_list.append(average_valid_loss)
                global_steps_list.append(global_step)

                # 만약 기존 것보다 성능이 높게 나왔다면 현재 모델 상태를 저장합니다.
                if best_valid_loss > average_valid_loss:
                    best_valid_loss = average_valid_loss
                    save_checkpoint(output_file_path + '/model.pt', model, optimizer, best_valid_loss)
                    save_metrics(output_file_path + '/metrics.pt', train_loss_list, valid_loss_list, global_steps_list)

    # 결과를 저장합니다.
    save_metrics(output_file_path + '/metrics.pt', train_loss_list, valid_loss_list, global_steps_list)

## evaluate 함수

해당 함수에서는 validation data를 이용하여 학습된 `model`을 평가합니다.

In [ ]:
def evaluate(model, device, valid_loader, loss_fn, running_loss, eval_every):
    # 학습 중 모델을 평가합니다.
    # 모델에게 학습이 아닌 평가를 할 것이라고 알립니다.
    model.eval()
    valid_running_loss = 0.

    # 학습이 아니기에 최적화를 하지 않는다는 환경을 설정합니다.
    with torch.no_grad():
        # validation 데이터를 읽습니다.
        for input_ids, attention_mask, labels in valid_loader:
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            # model을 함수처럼 호출하면 model에서 정의한 forward 함수가 실행됩니다.
            # 즉, 데이터를 모델에 집어넣어 forward방향으로 흐른 후 그 결과를 받습니다.
            output = model(input_ids, attention_mask)

            # validation 데이터의 loss, 즉 모델의 출력과 실제 데이터의 차이를 구합니다.
            loss = loss_fn(output, labels)
            valid_running_loss += loss.item()

    # 평균 loss를 계산합니다.
    average_train_loss = running_loss / eval_every
    average_valid_loss = valid_running_loss / len(valid_loader)

    return average_train_loss, average_valid_loss

## 그래프 시각화 함수

epoch에 따른 train loss와 validation loss 그래프를 그립니다.

In [ ]:
def draw_graph(output_file_path, device):
    train_loss_list, valid_loss_list, global_steps_list = load_metrics(output_file_path + '/metrics.pt', device)
    plt.plot(global_steps_list, train_loss_list, label='Train')
    plt.plot(global_steps_list, valid_loss_list, label='Valid')
    plt.xlabel('Global Steps')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig("train_valid_loss.png", bbox_inches='tight')

    Image('train_valid_loss.png')

## 모델 및 기록 저장 불러오기

In [ ]:
def save_checkpoint(save_path, model, optimizer, valid_loss):
    state_dict = {'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'valid_loss': valid_loss}

    torch.save(state_dict, save_path)


def load_checkpoint(load_path, model, optimizer, device):
    state_dict = torch.load(load_path, map_location=device)

    model.load_state_dict(state_dict['model_state_dict'])
    optimizer.load_state_dict(state_dict['optimizer_state_dict'])

    return state_dict['valid_loss']


def save_metrics(save_path, train_loss_list, valid_loss_list, global_steps_list):
    state_dict = {'train_loss_list': train_loss_list,
                  'valid_loss_list': valid_loss_list,
                  'global_steps_list': global_steps_list}

    torch.save(state_dict, save_path)


def load_metrics(load_path, device):
    state_dict = torch.load(load_path, map_location=device)

    return state_dict['train_loss_list'], state_dict['valid_loss_list'], state_dict['global_steps_list']

## 데이터 불러오기

데이터를 불러올 때 우리는 Bert Tokenizer를 사용합니다.

In [ ]:
# 데이터의 기본 형태에 대한 정보입니다.
output_file_path="./model/"
os.makedirs(output_file_path, exist_ok=True)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

root_path = "./"
train_file_path = root_path + "train.csv"
valid_file_path = root_path + "valid.csv"

max_len = 100
batch_size = 32

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# train, validation 데이터 csv 파일을 읽어옵니다.
train_loader = make_data_loader(train_file_path, tokenizer, max_len, batch_size)
valid_loader = make_data_loader(valid_file_path, tokenizer, max_len, batch_size)

## 모델 학습

`train` 함수를 이용하여 train data를 통해 모델 학습을 진행하세요.
**GPU를 사용할 수 있는 경우, 학습 시간은 약 10분 정도 소요됩니다.**

In [ ]:
# 앞서 정의한 BERTClassifier 클래스의 인스턴스를 만듭니다.
model = BERTClassifier().to(device)

# AdamW optimizier를 사용합니다.
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)

# 약 10분 정도 소요됩니다.
train(model, device, optimizer, train_loader, valid_loader, output_file_path, 5)  

## 결과 출력

학습을 수행하였으면 결과를 출력합니다.

In [ ]:
draw_graph(output_file_path, device)